In [1]:
import pandas as pd
import urllib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
comments = pd.read_csv('../data/attack_annotated_comments.tsv', sep = '\t', index_col = 0)
annotations = pd.read_csv('../data/attack_annotations.tsv',  sep = '\t')

In [3]:
# labels a comment as an atack if the majority of annoatators did so
labels = annotations.groupby('rev_id')['attack'].mean() > 0.5

In [4]:
# join labels and comments
comments['attack'] = labels

In [5]:
import string
# remove newline and tab tokens
comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
comments['comment'] = comments['comment'].apply(lambda x: x.lower())
comments['comment'] = comments['comment'].apply(lambda x: "".join(l for l in x if l not in string.punctuation))

In [6]:
comments.head()

,comment,year,logged_in,ns,sample,split,attack
rev_id,,,,,,,
37675,this is not creative those are the dictionar...,2002,False,article,random,train,False
44816,the term standard model is itself less npov...,2002,False,article,random,train,False
49851,true or false the situation as of march 2002...,2002,False,article,random,train,False
89320,next maybe you could work on being less conde...,2002,True,article,random,dev,False
93890,this page will need disambiguation,2002,True,article,random,train,False


In [ ]:
swear_words = "../swear_words.txt"
word_list = []
with open(swear_words) as f:
    word_list = f.read().splitlines()

In [ ]:
from textblob import TextBlob

def tokens(post):
    return TextBlob(post.decode('utf8')).words
    
def get_bad_word_count(post):
    x = tokens(post)
    count = 0
    bad_words = []
    for word in x:
        if word in word_list:
            count+=1
            bad_words.append(word)
    return count, str(bad_words)

comments['bad_word_count'],comments['bad_word_list']= zip(*comments['comment'].map(get_bad_word_count)) 

In [ ]:
def data_analysis(comments):
    total = len(comments)
    bully = len(comments[comments['attack']==1])
    swear = len(comments[comments['bad_word_count']>0])
    anonymous = len(comments[comments['logged_in']==0])
    swear_bully = len(comments[(comments['bad_word_count']>0) & (comments['attack']==1) ])
    anonymous_bully = len(comments[(comments['attack']==1) & (comments['logged_in']==0)])
    anonymous_swear = len(comments[(comments['bad_word_count']>0)  & (comments['logged_in']==0)])
    anonymous_bully_swear = len(comments[(comments['attack']==1) & (comments['bad_word_count']>0)  & (comments['logged_in']==0)])
    
    P_B = float(bully)/total
    P_S = float(swear)/total
    P_A = float(anonymous)/total
    P_B_S = float(swear_bully)/swear
    P_S_B = float(swear_bully)/bully
    P_B_A = float(anonymous_bully)/anonymous
    P_A_B = float(anonymous_bully)/bully
    P_S_A = float(anonymous_swear)/anonymous
    P_A_S = float(anonymous_swear)/total
    P_B_A_S = float(anonymous_bully_swear)/ anonymous_swear
    
    print("P(B): " + str(P_B))
    print("P(S): " + str(P_S))
    print("P(A): " + str(P_A))
    print("P(B|S): " + str(P_B_S))
    print("P(S|B): " + str(P_S_B))
    print("P(B|A): " + str(P_B_A))
    print("P(A|B): " + str(P_A_B))
    print("P(S|A): " + str(P_S_A))
    print("P(B|A & S): " + str(P_B_A_S))
    